In [1]:
!pip -qq install transformers datasets rouge_score nltk

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.4 MB/s eta 0:00:00


In [2]:
!pip install datasets transformers rouge-score pandas

In [3]:
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
from rouge_score import rouge_scorer
from torch.utils.data import DataLoader
from torch import nn
import torch
import re
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Loading pre-trained T5 model and tokenizer
model_name = 't5-base'  # Switching from 't5-small' to 't5-base' for better performance
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Text Preprocessing
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text

# summarizing text using T5
def summarize_text(article):
    input_text = f"summarize: {article}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True).to(device)
    summary_ids = model.generate(input_ids, max_length=150, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Evaluating using ROUGE and BLEU with Smoothing
def evaluate_model(dataset):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = {'rouge1': [], 'rouge2': [], 'rougeL': [], 'bleu': []}
    smooth = SmoothingFunction().method4  # Applying smoothing to BLEU

    for item in dataset:
        article = preprocess_text(item['Annual Reports'])
        gold_summary = preprocess_text(item['Gold Summaries'])

        if article:
            generated_summary = summarize_text(article)

            # ROUGE scores
            score = scorer.score(gold_summary, generated_summary)
            scores['rouge1'].append(score['rouge1'].fmeasure)
            scores['rouge2'].append(score['rouge2'].fmeasure)
            scores['rougeL'].append(score['rougeL'].fmeasure)

            # BLEU score with smoothing
            reference = gold_summary.split()  # Tokenizing gold summary
            candidate = generated_summary.split()  # Tokenizing generated summary
            bleu_score = sentence_bleu([reference], candidate, smoothing_function=smooth)
            scores['bleu'].append(bleu_score)

    avg_rouge1 = sum(scores['rouge1']) / len(scores['rouge1'])
    avg_rouge2 = sum(scores['rouge2']) / len(scores['rouge2'])
    avg_rougeL = sum(scores['rougeL']) / len(scores['rougeL'])
    avg_bleu = sum(scores['bleu']) / len(scores['bleu'])

    return avg_rouge1, avg_rouge2, avg_rougeL, avg_bleu

# Fine Tuning model
def fine_tune_model(train_data, epochs=5, learning_rate=5e-5):
    model.train()
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    loss_fn = nn.CrossEntropyLoss()

    dataloader = DataLoader(train_data, batch_size=8, shuffle=True)

    for epoch in range(epochs):
        total_loss = 0
        for batch in dataloader:
            optimizer.zero_grad()
            input_texts = batch['Annual Reports']
            target_summaries = batch['Gold Summaries']

            inputs = tokenizer(input_texts, return_tensors="pt", max_length=512, truncation=True, padding=True).input_ids.to(device)
            targets = tokenizer(target_summaries, return_tensors="pt", max_length=150, truncation=True, padding=True).input_ids.to(device)

            outputs = model(input_ids=inputs, labels=targets)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs} Loss: {total_loss/len(dataloader)}")

# Implementing Main function
def main():
    # Loading datasets
    train_data = load_dataset("ragha92/FNS_Summarization", split="train")  # Select first 1000 records
    validation_data = load_dataset("ragha92/FNS_Summarization", split="validation")  # Limit validation set to 1000
    test_data = load_dataset("ragha92/FNS_Summarization", split="test")  # No limit on the test set

    # Fine-tuning model
    fine_tune_model(train_data, epochs=5)

    # Evaluation on validation set for model selection
    print("Evaluating on the validation set...")
    avg_rouge1, avg_rouge2, avg_rougeL, avg_bleu = evaluate_model(validation_data)
    print(f"Validation ROUGE-1: {avg_rouge1}, ROUGE-2: {avg_rouge2}, ROUGE-L: {avg_rougeL}, BLEU: {avg_bleu}")

    # Final evaluation on test set
    print("Evaluating on the test dataset...")
    avg_rouge1, avg_rouge2, avg_rougeL, avg_bleu = evaluate_model(test_data)
    print(f"Test Set ROUGE-1: {avg_rouge1}")
    print(f"Test Set ROUGE-2: {avg_rouge2}")
    print(f"Test Set ROUGE-L: {avg_rougeL}")
    print(f"Test Set BLEU: {avg_bleu}")

# Defining Main function
if __name__ == "__main__":
    main()


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/2060 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/257 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/259 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


KeyboardInterrupt: 